In [1]:
import pandas as pd
import numpy as np

In [2]:
a_price = pd.read_csv("data/price/a_stock_price.csv")
b_price = pd.read_csv("data/price/b_stock_price.csv")

In [3]:
a_price["Trddt"] = pd.to_datetime(a_price["Trddt"],format="%Y%m%d")
b_price["Trddt"] = pd.to_datetime(b_price["Trddt"],format="%Y%m%d")

In [4]:
shanghai_ab_list = pd.read_csv("data/ticker_list/shanghai_ab_list.csv")
shenzhen_ab_list = pd.read_csv("data/ticker_list/shenzhen_ab_list.csv")

In [5]:
ab_list = pd.concat([shanghai_ab_list,shenzhen_ab_list])

unique_id_list = ["id_"+str(idx) for idx in range(0,ab_list.shape[0])]
ab_list["unique_id"] = unique_id_list

### Create Price DataFrame

In [ ]:
a_price_merge = a_price.merge(ab_list[["a_ticker","unique_id"]],left_on=["Stkcd"],right_on=["a_ticker"])
b_price_merge = b_price.merge(ab_list[["b_ticker","unique_id"]],left_on=["Stkcd"],right_on=["b_ticker"])

a_price_id = a_price_merge[["Trddt","unique_id","Clsprc"]]
b_price_id = b_price_merge[["Trddt","unique_id","Clsprc"]]

a_price_id.columns = ["Trddt","unique_id","a_price"]
b_price_id.columns = ["Trddt","unique_id","b_price"]

price_df = a_price_id.merge(b_price_id,on=["Trddt","unique_id"])

In [ ]:
price_df.to_csv("ab_stock_cross_section_new.csv")

### Create Asset Information

In [ ]:
a_industry = a_price_merge[["unique_id","Indcd","Nindcd","Nnindcd"]].drop_duplicates()
b_industry = b_price_merge[["unique_id","Indcd","Nindcd","Nnindcd"]].drop_duplicates()

In [ ]:
a_industry.to_csv("data/unique_id_industry_info.csv")

### Create Fundamental Information

In [ ]:
income_df = pd.read_excel("data/CSMAR_income/FS_Comins.xlsx")

In [ ]:
asset_df = pd.read_excel("data/CSMAR_asset/FS_Combas.xlsx")

In [ ]:
income_df["Accper"] = pd.to_datetime(income_df["Accper"])
asset_df["Accper"] = pd.to_datetime(asset_df["Accper"])

In [ ]:
income_df = income_df[income_df["Typrep"]=="A"]
asset_df = asset_df[asset_df["Typrep"]=="A"]

In [ ]:
asset_df = asset_df.merge(ab_list[["a_ticker","unique_id"]],left_on=["Stkcd"],right_on=["a_ticker"])

In [ ]:
income_df = income_df.merge(ab_list[["a_ticker","unique_id"]],left_on=["Stkcd"],right_on=["a_ticker"])

In [ ]:
asset_df = asset_df[~(asset_df["unique_id"].isna())]

In [ ]:
income_df = income_df[~(income_df["unique_id"].isna())]

In [ ]:
asset_df.to_csv("data/asset_df.csv")

In [ ]:
income_df.to_csv("data/income_df.csv")

In [6]:
share_df = pd.read_excel("data/CSMAR_shares/TRD_Capchg.xlsx")

In [7]:
share_df["股本变动日期"] = pd.to_datetime(share_df["股本变动日期"])

In [8]:
share_df.columns = ["Stkcd","Trddt","股本变动类型","A股流通股数"]

In [9]:
merge_df = a_price.merge(share_df,on=["Stkcd","Trddt"],how="left")

In [10]:
share_df = merge_df[["Stkcd","Trddt","A股流通股数","Clsprc"]]

In [11]:
share_df = share_df.set_index("Trddt")

In [12]:
share_df = share_df[["Stkcd","A股流通股数"]]

In [13]:
def fillna(x):
    
    return x["A股流通股数"].fillna(method="ffill")

In [14]:
share_df = share_df.groupby("Stkcd").apply(fillna)

In [15]:
share_df

Stkcd   Trddt     
2       2000-01-04            NaN
        2000-01-05            NaN
        2000-01-06            NaN
        2000-01-07            NaN
        2000-01-10    398711877.0
                         ...     
900957  2019-12-25            0.0
        2019-12-26            0.0
        2019-12-27            0.0
        2019-12-30            0.0
        2019-12-31            0.0
Name: A股流通股数, Length: 432886, dtype: float64

In [16]:
a_price_df = a_price[["Stkcd","Trddt","Clsprc"]]

In [17]:
a_price_df = a_price_df.set_index(["Stkcd","Trddt"])

In [18]:
market_cap_df = pd.concat([a_price_df,share_df],axis=1)

In [19]:
market_cap_df["market_cap"] = market_cap_df["Clsprc"] * market_cap_df["A股流通股数"]

In [22]:
index_df = market_cap_df.index.to_frame()

In [25]:
index_df["market_cap"] = market_cap_df["market_cap"]

In [27]:
index_df.index = range(0,index_df.shape[0])

In [37]:
merge_df = index_df.merge(ab_list[["a_ticker","unique_id"]],left_on="Stkcd",right_on="a_ticker",how="right")

In [38]:
market_cap_df = merge_df[["Trddt","unique_id","market_cap"]]

In [40]:
market_cap_df

,Trddt,unique_id,market_cap
0,2000-01-04,id_0,NaN
1,2000-01-05,id_0,NaN
2,2000-01-06,id_0,NaN
3,2000-01-07,id_0,NaN
4,2000-01-10,id_0,NaN
...,...,...,...
386882,2019-12-25,id_85,7.753837e+09
386883,2019-12-26,id_85,7.767783e+09
386884,2019-12-27,id_85,7.749188e+09
386885,2019-12-30,id_85,7.832863e+09


In [42]:
market_cap_df.to_csv("market_cap_df.csv")